# Analysis and implement of the paper: Variational Inference with Normalizing Flows

## Members: Abhimanyu Talwar, Julien Laasri, Feng Qian

# Problem Statement

Variational Inference has become a popular and even standard inferencing technique for large-scale and complex probabilistic models with increasingly larger data sets. Despite these successes, VI suffers from two main limitations:

* VI can only approximate complex posteriors using mean-field assumption with very easy and known distributions such as Normal Distribution, and this limits the ability of VI greatly. Often, VI is not able to deal with asymmetric posterior, which is the case for most posterior distributions.
* Since VI is trying to approximate posterior using symmetric distributions, the variance is very largely underestimated because the estimated posterior is "nudged in" from every dimension.

Variational Inference with Normalizing Flows is a technique trying to extend the choice of the approximating posterior from easy distributions to arbitrary distributions using normalizing flow, and so that could solve the two limitations of VI mentioned previously.

# Problem Context

For large-scale and complex probabilistic models with increasingly larger data sets, Variational Inference is almost the only choice. From the large-scale topic models of text(Hoffman et al., 2013, Stochastic variational inference) to semi-supervised classification (Kingma et al., 2014, Semi-supervised learning with deep generative models), VI is the only solution to inference the posterior of these complex models. MCMC methods are too inefficient to tackle with such models.

However, as mentioned before, normal VI suffers from being able to only choose from some simple mean-field distributions and underestimates the variance a lot. At the same time, researchers showed that more faithful posterior approximations do result in better performance.

So it is very clear that a technique expends the VI from the mean-field assumption approximation to an arbitrary approximation will be interesting and demanding.

# Existing work and literature review

Some great works are using VI and verifying the value of the VI, such as the Hoffman et al., 2013, (Stochastic variational inference), and the Kingma et al., 2014, (Semi-supervised learning with deep generative models). However, Turner et al., 2011, (Two problems with variational expectation maximization for time-series models) show clearly the two limitations mentioned above that VI has. At the same time, Minh et al., 2014, (Neural variational inference and learning in belief networks) show that more faithful posterior approximation does result in better results. 

There are earlier efforts to expand the range of choice of the approximating distributions. Earlier successful works are based on structured mean-field approximations that incorporate dependency within the approximate posterior, which is still very limited compares to an arbitrary approximation distribution. Some other works try to expand the range of choice by specifying the approximate posterior as a mixture model, for example, Jaakkola et al., 1998, (Improving the mean field approximation via the use of mixture distributions), and because of needing to evaluate the log-likelihood and needing to calculate gradients for the mixture model, the computation is too high for any larger model.

# Contribution of this paper

This paper is trying to propose a method that could expand the approximating distribution of VI from simple mean-field distributions to arbitrary distributions via a technique called the normalizing flow. The contribution of the paper is that tt successfully expands the approximating distribution to an arbitrary complex distribution using the normalizing flow.

# High-level technical idea

The high-level idea of the VI with NF is pretty simple: since all problems are caused by the simple approximating distribution, why don't we use a more complex distribution?

But the more complex distribution should have the following characteristic: It should not be too hard to calculate the expectation and conditional expectation of this distribution. The reason is simple: since you need to do the variational, you need to calculated related expectations and optimize the ELBO.

The users proposed to use the normalizing flow, which in simple words, is a special kind of transformation of a distribution that has some very good characteristics: computationally cheap, invertible, and easy to calculated related values. Also, it is proved that the normalizing flow could transform a Gaussian distribution to an arbitrary form, which solves the problem underlying a normal VI.

Using this normalizing flow, we can write out the new form of the ELBO, which is also called the negative free energy, and plug it into the regular VI algorithm.

In this way, we could do a VI inference with an arbitrary approximating distribution acquired using the normalizing flow.


# Detailed technical content

Firstly, we need to define what is a normalizing flow and how does the normalizing flow transforms a Gaussian distribution into an arbitrary distribution.

The basic transform is a smooth mapping $f$ with a inverse mapping $g$. The reason why it need to be invertable is that if we use $f$ to transform a variable $z$ with distribution $q(z)$, we need to be able to write out the density function of $z^{\prime}$. The transformed $z^{\prime}$ has the distribution:

$q\left(\mathbf{z}^{\prime}\right)=q(\mathbf{z})\left|\operatorname{det} \frac{\partial f^{-1}}{\partial \mathbf{z}^{\prime}}\right|=q(\mathbf{z})\left|\operatorname{det} \frac{\partial f}{\partial \mathbf{z}}\right|^{-1}$

Then, we could apply transformations consecutively to a variable $z_0$ $k$ times:

$z_k = f_{K}\left(f_{K-1}\left(\ldots f_{1}(x)\right)\right)$

,and it is easy to get the following equation using the equation we derived before:

$ \ln q_{K}\left(\mathbf{z}_{K}\right)=\ln q_{0}\left(\mathbf{z}_{0}\right)-\sum_{k=1}^{K} \ln \left|\operatorname{det} \frac{\partial f_{k}}{\partial \mathbf{z}_{k-1}}\right|$ 

Also, it's quick to have, according to the law of the unconscious statistation that:

$\mathbb{E}_{q_{K}}[h(\mathbf{z})]=\mathbb{E}_{q_{0}}\left[h\left(f_{K} \circ f_{K-1} \circ \ldots \circ f_{1}\left(\mathbf{z}_{0}\right)\right)\right]$

as long as the $h$ does not depend on $q$. This equation is important for us since we need it to write out the ELBO.

Not every transformation is computationally cheap. In order to avoid computing the Jacobian determinant which is computationally expensive, the paper proposed the following transformation that is computationally efficient:

$f(\mathbf{z})=\mathbf{z}+\mathbf{u} h\left(\mathbf{w}^{\top} \mathbf{z}+b\right)$

Where $w,u,b$ are free params and $h$ is a smooth non-linear function.

The Jacobian of the above transition is easy to calculate as:

$\psi(\mathbf{z})=h^{\prime}\left(\mathbf{w}^{\top} \mathbf{z}+b\right) \mathbf{w}$

$\left|\operatorname{det} \frac{\partial f}{\partial \mathbf{Z}}\right|=\left|\operatorname{det}\left(\mathbf{I}+\mathbf{u} \psi(\mathbf{z})^{\top}\right)\right|=\left|1+\mathbf{u}^{\top} \psi(\mathbf{z})\right|$

and this will simplify the normalizing flow greately by:

$z_k = f_{K}\left(f_{K-1}\left(\ldots f_{1}(x)\right)\right)$

$\ln q_{K}\left(\mathbf{z}_{K}\right)=\ln q_{0}(\mathbf{z})-\sum_{k=1}^{K} \ln | 1+\mathbf{u}_{k}^{\top} \psi_{k}\left(\mathbf{z}_{k-1}\right)$

Finally, it comes to ELBO. Once we could express the ELBO, we could plug everything into the usual VI algorithm.

Since we have:

$\begin{aligned} \mathcal{F}(\mathbf{x}) &=\mathbb{E}_{q_{\phi}(z | x)}\left[\log q_{\phi}(\mathbf{z} | \mathbf{x})-\log p(\mathbf{x}, \mathbf{z})\right] \\ &=\mathbb{E}_{q_{0}\left(z_{0}\right)}\left[\ln q_{K}\left(\mathbf{z}_{K}\right)-\log p\left(\mathbf{x}, \mathbf{z}_{K}\right)\right] \\ &=\mathbb{E}_{q_{0}\left(z_{0}\right)}\left[\ln q_{0}\left(\mathbf{z}_{0}\right)\right]-\mathbb{E}_{q_{0}\left(z_{0}\right)}\left[\log p\left(\mathbf{x}, \mathbf{z}_{K}\right)\right] \\ &-\mathbb{E}_{q_{0}\left(z_{0}\right)}\left[\sum_{k=1}^{K} \ln | 1+\mathbf{u}_{k}^{\top} \psi_{k}\left(\mathbf{z}_{k-1}\right)\right] \end{aligned}$

Now we are ready to use the transformed distribution and transformed ELBO for inference.


# Experiments implemented by the paper

Firstly, the paper compares the representative power between two forms of transitions: Normalizing flow and volume-preserving transformation (NICE). The paper used four artificial distirbutions and applied both VI-NF and VI-NICE on them. The result shows that VI-NF has better representative power and using fewer parameters. Also, this experiment shows that VI-NF has the ability to represent asymmetric posterior distributions.

Then, the paper did two classification task using the VI-NF. One experiment is done on the MINST handwritten picture data set and the other on the CIFAR-10 picture classification dataset. Both experiments show that the NF version of VI outperforms other kinds of VI including VI using the NICE and HVI. 

The experiment is able to demonstrate the exceptional ability and performance of the VI-NF. VI-NF can use arbitrary approximating distributions, can work on asymmetric posteriors and outperforms other variants of VI algorithms. 

However, the experiment is not perfect, and we have found some weaknesses in the design of the experiment. We will list these weaknesses in the next section.

# Our evaluation of this paper

This paper is technically sound. The proof and math of the paper are very logical and strict. Experiments on deep latent Gaussian models, the MINST data set, and the CIFAR-10 Image data set show that the technique outperforms traditional common variational inference techniques, which shows the practical ability of the VI with NF.

However, our team still thinks that the experiment part could be stronger in terms of:

1. The advantage of VI with NF (personally speaking) is not very big on the MNIST data compares to volume-preserving approaches.

1. We don't understand why the paper does not report results on the CIFAR-10 dataset by other methods. Very possibly, the VI with NF does not work very well on the image dataset, which brings questions to the particular usage of the technique on image-related tasks.

1. The paper claims that VI with NF could give a more accurate inference of the variance, but there is not an analysis of variance at all in the paper. We think that the experiment will be more completed if there could be more experiments analyzing improvement on a more accurate variance estimation.

1. The reimplementation of the paper is very hard. We did a lot of tuning on hyperparameters, and the model does not converge from time to time. We suspect that VI with NF is very hard to converge and the paper will be more completed if there is some analysis on the convergence and stability of this method.

# Code
All the code deliverables can be found in the two notebooks that come with this report. Notebook-1 contains pure Variational Inference which aims at approximating a posterior distribution using normalizing flows and Notebook-2 has the implementation of a full VAE using normalizing flows which includes an inference and a generative network.
